# MODULES

In [1]:
from PIL import Image as img
import os
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import smtplib
from email.message import EmailMessage

from process_data import *
from model import *

In [2]:
def email_alert(subject, body , to):
    msg= EmailMessage()
    msg.set_content(body)
    msg['subject'] = subject
    msg['to'] = to
    

    user = "jyotikrish669@gmail.com"#email account
    msg['from']= user
    password = "bemcgjrfmlzwztgo"#email id password
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(user, password)
    server.send_message(msg)
    
    
    server.quit()

In [3]:

def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")



def save_frame(video_path,save_dir,gap = 15,width=250,height=250):
    name = video_path.split("\\")[-1].split(".")[0]
    # name = video_path
    save_path = os.path.join(save_dir , name)
    create_dir(save_path)
    gun_path = os.path.join(save_dir , name, "detected_guns")
    create_dir(gun_path)
    cap=cv2.VideoCapture(video_path)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break
        if idx % gap ==0:#gap reduces the no. of frames per sec
            cv2.imwrite(f"{save_path}/{name}{idx}.jpg", frame)
            
            gun_img = img.open(f"{save_path}/{name}{idx}.jpg")
            gun_img = gun_img.resize((width,height))

            gun = np.array(gun_img)
            gun = gun.flatten().reshape(width*height*3,1)
            
            theta = np.genfromtxt("theta.csv",delimiter=',').reshape(width*height*3+1,1)
            gun = np.vstack((gun,[[1]]))
            if predict(gun/255,theta)==1:
                cv2.imwrite(f"{save_path}/detected_guns/{name}{idx}.jpg", frame)
                email_alert("Criminal Alert",f"suspicious criminal activity detected. Gun path- {save_path}/detected_guns/{name}{idx}.jpg","guddu.jijnasu999@gmail.com")
        
        idx+=1




In [5]:
def extract_frames(videos_paths="captures",save_dir="frames",gap=15):
    all_videos = glob.glob(videos_paths+"/*")
    all_extracted = glob.glob(save_dir+"/*")
    for i,video in enumerate(all_extracted):
        all_extracted[i] = video.split("\\")[-1]
        
    for i,video in enumerate(all_videos):
        if video.split("\\")[-1].split(".")[0] in all_extracted:
            all_videos[i]=0
        
    
    while 0 in all_videos:
        all_videos.remove(0)
    for path in all_videos:
        print(path.split("\\")[-1])
        save_frame(path, save_dir,gap=gap)

In [7]:
extract_frames(gap=20)